In [ ]:
!cp -r /content/drive/My\ Drive/Capstone/dataset.zip /content/

In [ ]:
!unzip dataset.zip

In [ ]:
!mv /content/content/dataset /content/

In [ ]:
!rm -rf /content/content
!rm -rf /content/dataset.zip

In [ ]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
#Removing data, taking only 1k images for both class
import os
i=0
path='/content/dataset/Non Dove/'
for r,d,f in os.walk(path):
  for file in f:
    if i>1000:
      os.remove(r+file)
    i+=1
    

In [ ]:
data_dir='/content/dataset'
img_width=img_height=128
batch_size=100

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='binary',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    subset='validation'
    )

Found 1602 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
input_shape = (128,128,3) 
num_classes = 1

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.7))
model.add(Flatten())
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-8, amsgrad=False),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 124, 124, 32)      2432      
_________________________________________________________________
activation_6 (Activation)    (None, 124, 124, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 58, 58, 64)        51264     
_________________________________________________________________
activation_7 (Activation)    (None, 58, 58, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 29, 29, 64)       

In [ ]:
training=model.fit_generator(
  train_generator,
  epochs = 10,
  validation_data = validation_generator,
  validation_steps=10)

In [ ]:
from keras.applications import VGG16

In [ ]:
vgg = VGG16(input_shape = [128,128] + [3], weights = 'imagenet', include_top = False)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
from keras.models import Model
from keras.layers import Flatten, Dense

x = Flatten()(vgg.output)
#x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)      

In [ ]:
from keras.applications.vgg16 import preprocess_input

In [ ]:
train_generator.class_indices

{'Dove': 0, 'Non Dove': 1}

In [ ]:
history = model.fit_generator(train_generator,
                   epochs = 10, 
                   validation_data = validation_generator,
                   validation_steps = 100)  

Epoch 1/10
17/17 [==============================] - ETA: 0s - loss: 7.6246 - accuracy: 0.5000 WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 100 batches). You may need to use the repeat() function when building your dataset.


17/17 [==============================] - 840s 49s/step - loss: 7.6246 - accuracy: 0.5000 - val_loss: 7.6246 - val_accuracy: 0.5000
Epoch 2/10
17/17 [==============================] - 630s 37s/step - loss: 7.6246 - accuracy: 0.5000
Epoch 3/10
17/17 [==============================] - 628s 37s/step - loss: 7.6246 - accuracy: 0.5000
Epoch 4/10
17/17 [==============================] - 628s 37s/step - loss: 7.6246 - accuracy: 0.5000
Epoch 5/10
17/17 [==============================] - 632s 37s/step - loss: 7.6246 - accuracy: 0.5000
Epoch 6/10
17/17 [==============================] - 640s 38s/step - loss: 7.6246 - accuracy: 0.5000
Epoch 7/10
17/17 [==============================] - 649s 38s/step - loss: 7.6246 - accuracy: 0.5000
Epoch 8/10
17/17 [==============================] - 645s 38s/step - loss: 7.6246 - accuracy: 0.5000
Epoch 9/10
17/17 [==============================] - 644s 38s/step - loss: 7.6246 - accuracy: 0.5000
Epoch 10/10
17/17 [==============================] - 646s 38s/step - 

In [ ]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from keras.preprocessing import image
import matplotlib.image as mpimg

file_name='test.png'
plt.figure(figsize=(10,10))

test_image=image.load_img(file_name,target_size=(128,128))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=model.predict(test_image)
print(result)

[[1.]]


<Figure size 720x720 with 0 Axes>

In [ ]:
index=train_generator.class_indices

In [ ]:
index

{'Dove': 0, 'Non Dove': 1}

In [ ]:
print(result)

[[1.]]


In [ ]:
def get_key(val): 
    for key, value in index.items(): 
         if val == value: 
             return key 
  
    return "key doesn't exist"

In [ ]:
print(get_key(result[0][0]))

Non Dove
